In [1]:
# rm(list=ls())
require(data.table)
require(MASS)
require(ggplot2)
# require(foreach)
require(gridExtra)
require(stringr)
require(matrixStats)
require(parallel)
require(abind)
require(rlist)


parallel::detectCores()
## Resizing notebook plot space
options(repr.plot.width=16, repr.plot.height=9)

Loading required package: data.table
Loading required package: MASS
Loading required package: ggplot2
Loading required package: gridExtra
Loading required package: stringr
Loading required package: matrixStats
matrixStats v0.51.0 (2016-10-08) successfully loaded. See ?matrixStats for help.
Loading required package: parallel
Loading required package: abind
Loading required package: rlist


[1] 40

## Bring in LDI, TFR and Education

In [2]:
ldi <- data.table(fread('/share/forecasting/data/LDIpc/national_LDIpc_prepped_20161207.csv'))
head(ldi)

Read 12090 rows and 1008 (of 1008) columns from 0.112 GB file in 00:00:03


location_id,ihme_loc_id,year_id,age_group_id,sex_id,LDIpc_1,LDIpc_2,LDIpc_3,LDIpc_4,LDIpc_5,⋯,LDIpc_994,LDIpc_995,LDIpc_996,LDIpc_997,LDIpc_998,LDIpc_999,LDIpc_1000,rt_mean,rt_upper,rt_lower
6,CHN,1979,22,3,1015.737,1015.737,1015.737,1015.737,1015.737,⋯,1015.737,1015.737,1015.737,1015.737,1015.737,1015.737,1015.737,1015.737,1015.737,1015.737
6,CHN,1980,22,3,1028.146,1028.146,1028.146,1028.146,1028.146,⋯,1028.146,1028.146,1028.146,1028.146,1028.146,1028.146,1028.146,1028.146,1028.146,1028.146
6,CHN,1981,22,3,1044.316,1044.316,1044.316,1044.316,1044.316,⋯,1044.316,1044.316,1044.316,1044.316,1044.316,1044.316,1044.316,1044.316,1044.316,1044.316
6,CHN,1982,22,3,1075.913,1075.913,1075.913,1075.913,1075.913,⋯,1075.913,1075.913,1075.913,1075.913,1075.913,1075.913,1075.913,1075.913,1075.913,1075.913
6,CHN,1983,22,3,1114.356,1114.356,1114.356,1114.356,1114.356,⋯,1114.356,1114.356,1114.356,1114.356,1114.356,1114.356,1114.356,1114.356,1114.356,1114.356
6,CHN,1984,22,3,1171.506,1171.506,1171.505,1171.505,1171.505,⋯,1171.505,1171.505,1171.506,1171.505,1171.505,1171.505,1171.505,1171.506,1171.506,1171.505


In [3]:
edu <- data.table(fread("/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/output_data/20161121_GBD2016prelim_95+_raked_2/country_year_A_S_T_collapsed.csv"))
head(edu)

Read 17321 rows and 1005 (of 1005) columns from 0.274 GB file in 00:00:05


location_id,year_id,draw0,draw1,draw2,draw3,draw4,draw5,draw6,draw7,⋯,draw993,draw994,draw995,draw996,draw997,draw998,draw999,mean,lower,upper
10,1950,1.336208,1.678651,1.666961,1.321678,1.726451,2.100176,1.644592,1.935168,⋯,1.805843,1.324314,1.176254,1.550790,2.723640,1.806279,2.049624,1.645919,1.068804,2.630491
10,1951,1.363233,1.692118,1.694677,1.350223,1.766093,2.145282,1.688327,1.959271,⋯,1.823794,1.364587,1.208113,1.570845,2.754895,1.836829,2.107365,1.675652,1.087175,2.682010
10,1952,1.391230,1.705612,1.723597,1.380729,1.806876,2.189708,1.732409,1.983210,⋯,1.841873,1.406279,1.241448,1.591174,2.785191,1.867273,2.165253,1.706197,1.111213,2.737125
10,1953,1.420162,1.719183,1.753810,1.413282,1.848761,2.233337,1.776700,2.007246,⋯,1.860134,1.449185,1.276323,1.611863,2.814625,1.897611,2.223214,1.737582,1.135241,2.791019
10,1954,1.449966,1.732866,1.785342,1.447930,1.891650,2.276010,1.821005,2.031560,⋯,1.878600,1.493041,1.312772,1.633004,2.843222,1.927806,2.281070,1.769783,1.160084,2.835183
10,1955,1.480594,1.746735,1.818158,1.484754,1.935420,2.317627,1.865190,2.056326,⋯,1.897400,1.537597,1.350822,1.654735,2.871037,1.957871,2.338589,1.802792,1.190560,2.875104


### The first step is to compute the correlation of in-sample data. Let's subset those out first

In [36]:
ldi_insample <- ldi[year_id<=2015 , .SD, .SDcols = c("location_id", "year_id", "rt_mean")]
colnames(ldi_insample) <- c("location_id", "year_id", "mean")
edu_insample <- edu[year_id<=2015 & year_id>=1979 , .SD, .SDcols = c("location_id", "year_id", "mean")]
colnames(edu_insample) <- c("location_id", "year_id", "mean")

### Merge and compute correlation

In [38]:
insample_data_merged <- merge(x=ldi_insample, y=edu_insample, by=c("location_id", "year_id" ))
tail(insample_data_merged)

location_id,year_id,mean.x,mean.y
522,2010,3284.054,6.045872
522,2011,3459.080,6.046340
522,2012,3539.397,6.049182
522,2013,3625.256,6.054970
522,2014,3699.735,6.064234
522,2015,3761.979,6.077547


### Compute the correlation

In [61]:
print(corr_mat<-cor(insample_data_merged[, .(mean.x, mean.y)], method = "spearman"))

          mean.x    mean.y
mean.x 1.0000000 0.7328612
mean.y 0.7328612 1.0000000


### Neal's univariate copula function

In [41]:
draw2Dcopula <- function(X, cor_mat, df_return = F){
  L <- dim(X)[2]
  D <- dim(X)[3]
  Xsum <- apply(X, c(2, 3), sum)
  mvdat <- mvrnorm(n=D, mu=0 * 1:L, Sigma=cor_mat, empirical=TRUE)
  ranks <- apply(mvdat, 2, rank, ties.method="first")
  sortedXsim <- apply(Xsum, 1, function(x) sort(x, index.return=TRUE)$ix)
  sortedX <- X
  for(i in 1:L){
    sortedX[,i,] <- X[,i,sortedXsim[,i]]
  }
  Xcorr <- sortedX
  for(i in 1:L){
    Xcorr[,i,] <- sortedX[,i,ranks[,i]]
  }
  if (df_return==T) {
    return(data.frame(reshape2::melt(Xcorr)))
    }
      else {
          Xcorr
      }
}

### Merge the draws of LDI and Education, reshape super long and convert to arrays

In [54]:
ldi_fc <- ldi[, .SD, .SDcols = c("location_id", "year_id", paste0("LDIpc_", c(1:1000)))]
ldi_fc <- ldi_fc[, variable := "LDIpc"]
ldi_fc <- setcolorder(ldi_fc, c("location_id", "year_id", "variable", paste0("LDIpc_", c(1:1000))) )
colnames(ldi_fc) <-  c("location_id", "year_id", "variable", paste0("draw_", c(1:1000)))
ldi_fc <- melt(ldi_fc, id.vars=c("location_id","year_id", "variable"), variable.name = "draw_num", value.name = "data")
head(ldi_fc)

location_id,year_id,variable,draw_num,data
6,1979,LDIpc,draw_1,1015.737
6,1980,LDIpc,draw_1,1028.146
6,1981,LDIpc,draw_1,1044.316
6,1982,LDIpc,draw_1,1075.913
6,1983,LDIpc,draw_1,1114.356
6,1984,LDIpc,draw_1,1171.506


In [55]:
edu_fc <- edu[year_id>=1979, .SD, .SDcols = c("location_id", "year_id", paste0("draw", c(0:999)))]
edu_fc <- edu_fc[, variable := "EDU"]
edu_fc <- setcolorder(edu_fc, c("location_id", "year_id", "variable", paste0("draw", c(0:999))) )
colnames(edu_fc) <-  c("location_id", "year_id", "variable", paste0("draw_", c(1:1000)))
edu_fc <- melt(edu_fc, id.vars=c("location_id","year_id", "variable"), variable.name = "draw_num", value.name = "data")
head(edu_fc)

location_id,year_id,variable,draw_num,data
10,1979,EDU,draw_1,2.534496
10,1980,EDU,draw_1,2.588469
10,1981,EDU,draw_1,2.655124
10,1982,EDU,draw_1,2.725430
10,1983,EDU,draw_1,2.798605
10,1984,EDU,draw_1,2.873696


In [59]:
all_data_binded <- rbind(ldi_fc, edu_fc)
countries <- unique(edu_fc[, location_id])

system.time(myarray2<- lapply(countries, 
                                function(x) reshape2::acast(all_data_binded[location_id == x,], 
                                                      location_id ~  year_id ~ variable ~ draw_num, 
                                                      value.var = "data")))
myarray2 <- abind(myarray2, along=1)
str(myarray2)

   user  system elapsed 
 28.452   0.051  28.485 

 num [1:191, 1:62, 1:2, 1:1000] 2.53 11.35 12.45 3.18 4.12 ...
 - attr(*, "dimnames")=List of 4
  ..$ : chr [1:191] "10" "101" "102" "105" ...
  ..$ : chr [1:62] "1979" "1980" "1981" "1982" ...
  ..$ : chr [1:2] "EDU" "LDIpc"
  ..$ : chr [1:1000] "draw_1" "draw_2" "draw_3" "draw_4" ...


### Copula'in time

In [77]:
head(all_data_binded)

location_id,year_id,variable,draw_num,data
6,1979,LDIpc,draw_1,1015.737
6,1980,LDIpc,draw_1,1028.146
6,1981,LDIpc,draw_1,1044.316
6,1982,LDIpc,draw_1,1075.913
6,1983,LDIpc,draw_1,1114.356
6,1984,LDIpc,draw_1,1171.506


In [78]:
system.time(Xcorr_array <- lapply(countries, 
                                     function(x) cbind(x, draw2Dcopula(myarray2[paste0(x),,,], corr_mat , df_return = T)) ))
                                        
test2 <- list.stack(Xcorr_array, data.table=T)
colnames(test2) <- c("location_id", "year_id", "variable", "draw_num", "data")
tail(test2)                  

   user  system elapsed 
  9.536   0.000   9.531 

location_id,year_id,variable,draw_num,data
99,2035,LDIpc,draw_1000,32011.65
99,2036,LDIpc,draw_1000,32989.66
99,2037,LDIpc,draw_1000,33747.96
99,2038,LDIpc,draw_1000,34411.53
99,2039,LDIpc,draw_1000,34864.70
99,2040,LDIpc,draw_1000,35439.63


In [80]:
### Stack into a DT
test3 <- dcast(test2, location_id + year_id + variable ~ draw_num, value.var = c("data"))
head(test3)

location_id,year_id,variable,draw_1,draw_2,draw_3,draw_4,draw_5,draw_6,draw_7,⋯,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,draw_1000
6,1979,EDU,5.642348,5.447712,6.016524,6.115722,5.747208,5.828471,6.235271,⋯,5.823572,5.924700,5.696795,5.633075,5.776423,5.156318,5.379345,5.751900,6.227927,6.251851
6,1979,LDIpc,1015.736700,1015.736700,1015.736700,1015.736700,1015.736700,1015.736700,1015.736700,⋯,1015.736700,1015.736700,1015.736700,1015.736700,1015.736700,1015.736700,1015.736700,1015.736700,1015.736700,1015.736700
6,1980,EDU,5.772344,5.589353,6.116313,6.207940,5.872911,5.946150,6.329412,⋯,5.924677,6.016976,5.819375,5.751238,5.874576,5.290668,5.513045,5.864611,6.325511,6.329776
6,1980,LDIpc,1028.146400,1028.146400,1028.146400,1028.146400,1028.146400,1028.146400,1028.146400,⋯,1028.146400,1028.146400,1028.146400,1028.146400,1028.146400,1028.146400,1028.146400,1028.146400,1028.146400,1028.146400
6,1981,EDU,5.904181,5.733863,6.218437,6.305171,5.999061,6.065895,6.428797,⋯,6.031810,6.113720,5.944626,5.872336,5.977856,5.430139,5.646768,5.980017,6.427853,6.414947
6,1981,LDIpc,1044.315900,1044.315900,1044.315900,1044.315900,1044.315900,1044.315900,1044.315900,⋯,1044.315900,1044.315900,1044.315900,1044.315900,1044.315900,1044.315900,1044.315900,1044.315900,1044.315900,1044.315900


In [81]:
LDI_sorted <- test3[variable=="LDIpc", .SD, .SDcols = c("location_id", "year_id", paste0("draw_", c(1:1000)))]
colnames(LDI_sorted) <- c("location_id", "year_id", paste0("LDIpc_", c(1:1000)))
EDU_sorted <- test3[variable=="EDU", .SD, .SDcols = c("location_id", "year_id", paste0("draw_", c(1:1000)))]
colnames(EDU_sorted) <- c("location_id", "year_id", paste0("draw", c(0:999)))

In [82]:
## Save out
fwrite(LDI_sorted, "/ihme/forecasting/data/covariates/ldi_per_capita/national_LDIpc_corrd_with_EDU_20170210.csv")
fwrite(EDU_sorted, "/ihme/forecasting/data/covariates/education/national_EDU_corrd_with_LDIpc_20170210")